# D212 PA 3 code - Doug Haunsperger

## Data Preparation

### Do initial package import and data read

In [5]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

# Install package if needed for environment
#import sys
#!{sys.executable} -m pip install mlxtend

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

df = pd.read_csv('medical_market_basket_nonblank.csv')
#view first 5 rows
df.head(1)

,Presc01,Presc02,Presc03,Presc04,Presc05,Presc06,Presc07,Presc08,Presc09,Presc10,Presc11,Presc12,Presc13,Presc14,Presc15,Presc16,Presc17,Presc18,Presc19,Presc20
0,abilify,atorvastatin,folic acid,naproxen,losartan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Convert DF to list of lists for transactionEncoder, dropping NaN values
# Code ref: https://stackoverflow.com/a/58436257 (Boston, 2019).

df_out = df.apply(lambda x: list(x.dropna().values), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_out).transform(df_out)
enc_df = pd.DataFrame(te_ary, columns=te.columns_)

enc_df.head()

,Duloxetine,Premarin,Yaz,abilify,acetaminophen,actonel,albuterol HFA,albuterol aerosol,alendronate,allopurinol,...,trazodone HCI,triamcinolone Ace topical,triamterene,trimethoprim DS,valaciclovir,valsartan,venlafaxine XR,verapamil SR,viagra,zolpidem
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


### Output cleaned and encoded data set

In [3]:
enc_df.to_csv('enc_medical_markbask.csv', index=False)

## Data Analysis

### Perform market basket analysis

In [4]:
# Define minimum number of patients using a drug or drug combo to consider

min_patients = 5
tot_patients = enc_df.shape[1]


frequent_itemsets = apriori(enc_df, min_support= min_patients/tot_patients, use_colnames=True, max_len = 3)
rules = association_rules(frequent_itemsets, metric="lift",  min_threshold = 1)

display(frequent_itemsets.sort_values(by='support', ascending=False))

print("Rules identified: ", len(rules))
display(rules.sort_values(by='zhangs_metric', ascending=False))


,support,itemsets
1,0.238368,(abilify)
5,0.179709,(amphetamine salt combo xr)
7,0.174110,(carvedilol)
18,0.170911,(glyburide)
12,0.163845,(diazepam)
22,0.132116,(losartan)
6,0.129583,(atorvastatin)
21,0.098254,(lisinopril)
25,0.095321,(metoprolol)
13,0.095054,(doxycycline hyclate)


Rules identified:  8


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(abilify),(atorvastatin),0.238368,0.129583,0.047994,0.201342,1.553774,0.017105,1.089850,0.467950
3,(atorvastatin),(abilify),0.129583,0.238368,0.047994,0.370370,1.553774,0.017105,1.209650,0.409465
4,(abilify),(carvedilol),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
5,(carvedilol),(abilify),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
6,(abilify),(diazepam),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197
7,(diazepam),(abilify),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965
0,(abilify),(amphetamine salt combo xr),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562
1,(amphetamine salt combo xr),(abilify),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
